# Custom Models, Layers, and Loss Functions with TensorFlow 
### *Week: 1*

#### DeepLearningAI

## Description

* Compare Functional and Sequential APIs, discover new models you can build with the Functional API, and build a model that produces multiple outputs including a Siamese network.
* Build custom loss functions (including the contrastive loss function used in a Siamese network) in order to measure how well a model is doing and help your neural network learn from training data. 
* Build off of existing standard layers to create custom layers for your models, customize a network layer with a lambda layer, understand the differences between them, learn what makes up a custom layer, and explore activation functions. 
* Build off of existing models to add custom functionality, learn how to define your own custom class instead of using the Functional or Sequential APIs, build models that can be inherited from the TensorFlow Model class, and build a residual network (ResNet) through defining a custom model class. 


The DeepLearning.AI TensorFlow: Advanced Techniques Specialization introduces the features of TensorFlow that provide learners with more control over their model architecture and tools that help them create and train advanced ML models.  

This Specialization is for early and mid-career software and machine learning engineers with a foundational understanding of TensorFlow who are looking to expand their knowledge and skill set by learning advanced TensorFlow features to build powerful models.

## Sequential Model Example

```Python
seq_model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
        ])
```

* Sequential method return the model.
* The first layer is **flatten**, it takes a *image* for *28x28* to flat it.
* The second layer is **Dense** layer, this layer has 128 dense neuron and each activation function is **relu** function.
* The last layer is  **Dense** layer, it has 10 neuron, which means our output has 10 different categories, and it's activation function is **softmax** function.

## Functional API

|1 -> |2 -> |3|
|--|--|--|
|**Input**|**Layers**|**Model**|
|Define input to the model.| Define a set of interconnected layers on the input.|Define the model using the input and output layers.|

## Defining the Input

```Python
from tensorflow.keras.layers import Input

...

input = Input(shape(28,28)) # Our example is Mnist dataset.
```

## Defining the layers

```Python
from tensorflow.keras.layers import Dense, Flatten

...

x = Flatten()(input)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
```

## Defining the Model

```Python
from tensorflow.leras.models import Model

...

func_model = Model(inputs=input, outputs=predictions)
```

### [The first lab notebook](C1_W1_Lab_1_functional-practice.ipynb)

## Declaring and Stacking layers

```Python

def build_model_with_functional():
    from tensorflow.keras.models import Model
    
    input_layer = tf.keras.Input(shape=(28, 28))
    flatten_layer = tf.keras.layers.Flatten()(input_layer)
    first_dense = tf.keras.layers.Dense(128, activation=tf.nn.relu)(flatten_layer) # This layer should follow the flatten_layer
    
    # Also we could make other stlye like;
    
    """
    first_dense = tf.keras.layers.Dense(128, activation=tf.nn.relu)
    first_dense(flatten_layer)
    """
    
    output_layer = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(first_dense)
    func_model = Model(inputs=input_layer, output=output_layer)
    # We may have also multiple inputs and multiple outputs
    
    return func_model

```

## Branching Models

![inception_models](inception_model.png)

*It's not a direct linear model from layer to layer.*

#### Pseudocode will be like that

```Python
layer1 = Dense(32)
layer2_1 = Dense(32)(layer1)
layer2_2 = Dense(32)(layer1)
layer2_3 = Dense(32)(layer1)
layer2_4 = Dense(32)(layer1)

merge = Concatenate([layer2_1, layer2_2, layer2_3, layer2_4])

# Multiple inputs and outputs example
func_model = Model(inputs=[input1, input2], outputs=[output1, output2])
```

![layers_image](layer_1_to_4.png)

#### [*A Simple Guide to the Versions of the Inception Network*](https://towardsdatascience.com/a-simple-guide-to-the-versions-of-the-inception-network-7fc52b863202)

## Creating a Multi-Output Model

[*Energy Effiency Dataset Link*](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency)

The data has *8 features* and *2 labels*.

|Features|Labels|
|--|--|
|Relative Compactness|Heating Load|
|Surface Area|Cooling Load|
|Wall Area||
|Roof Area||
|Overall Height||
|Orientation||
|Glazing Area||
|Glazing Area Distribution||

![Multiple Output](multiple_output.png)

```Python
input_layer = Input(shape=(len(train.columns),))
first_dense = Dense(Units='128', activation='relu')(input_layer)
second_dense = Dense(units='128', activation='relu')(first_dense)

y1_output = Dense(unit'1', name='y1_output')(second_dense)

third_dense = Dense(units='64', activation='relu')(second_dense)

y2_output = Dense(units='1', name='y2_output')(third_dense)

# Define the model with input layer and a list of output layers
model = Model(inputs=input_layer, outputs=[y1_output, y2_output])
```

### [The second lab notebook](C1_W1_Lab_2_multi-output.ipynb)

## Siamese network: a Multiple-Input model

![Siamese_network](siamese_network.png)

**Desc**: You have two inputs, in this case two input images, which are processed with the two sub-networks that have the same base neural network architecture. We can measure the Euclidean distance between the output vectors of these networks to predict how similar these two input examples are or how different they are. 

#### *Siamese network references* 
1. [ Learning a Similarity Metric Discriminatively, with Application to Face Verification (Chopra, Hadsell, & LeCun, 2005)](http://yann.lecun.com/exdb/publis/pdf/chopra-05.pdf) 
2. [Similarity Learning with (or without) Convolutional Neural Network (Chatterjee & Luo, n.d.)](http://slazebni.cs.illinois.edu/spring17/lec09_similarity.pdf)

## Coding a Multi-Input Siamese network

**Defining the Base Network**

```Python
def initialize_base_network():
    input = Input(shape=(28, 28),)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)
    x = Dense(128, activation='relu')(x)
    return Model(inputs= input, outputs= x)
```

### Re-using the base network

```Python
input_a = Input(shape(28, 28),)
input_b = Input(shape(28, 28),)

vect_output_a = base_network(input_a)
vect_output_b = base_network(input_b)
```

![Multi_input](siamese_multi_input.png)

### Output functions

```Python
def euclidean_disance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis= 1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return(shape1[0], 1)


output = Lambda(euclidean_distance, output_shape = eucl_dist_output_shape)([vec_output_a, vect?output_b])

```

### Defining the final model

```Python
model = Model([input_a, input_b], output)

rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms) # contrastive_loss is custom loss function. It's created for our model.
```


### Train the Model

```Python
model.fit([tr_pairs[:,0], tr_pairs[:,1]], tr_y # Training data -> tr
           epochs=20,
          batch_size=128,
          validation_data=([ts_pairs[:,0], ts_pairs[:,1]], rs_y)
         )
```

**tr_pairs[:,0]** feeds the first, left column input of our model.

**tr_pairs[:,1]** feeds the seconds, right column input of our model.

**tr_y** is the similarity value, 1 is similar and 0 is different each other.


![similarity_output](similarity.png)

### [The third lab notebook](C1_W1_Lab_3_siamese-network.ipynb)